<a href="https://colab.research.google.com/github/sahandv/science_science/blob/master/FastText_embedding_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FASTTEXT EMBEDDING


## Init

Local OR Colab?

In [ ]:
# datapath = '/home/sahand/GoogleDrive/Data/' # Local
datapath = 'drive/My Drive/Data/' # Remote

### Clone Project Git Repo

In [ ]:
!rm -rf 'science_science'
username = "sahandv"#@param {type:"string"}
# password = ""#@param {type:"string"} 

!git clone https://github.com/$username/science_science.git
!ls

Cloning into 'science_science'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 856 (delta 123), reused 83 (delta 26), pack-reused 660
Receiving objects: 100% (856/856), 87.67 MiB | 25.61 MiB/s, done.
Resolving deltas: 100% (486/486), done.
drive  sample_data  science_science


### Mount Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
# Check files!
!ls 'drive/My Drive/Data-Permenant/FastText-crawl-300d-2M-subword'

crawl-300d-2M-subword.bin  crawl-300d-2M-subword.vec


### Install requirements

In [ ]:
!pip install fasttext
!pip install -r 'science_science/requirements.txt'
!pip install gensim==3.8.1

     |████████████████████████████████| 71kB 1.8MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3016226 sha256=c33bf0a710561da56bf86896292ed2b0b6f3c8ada8d6575a81fbc8e27f2e6ddb
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext
     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 552kB 16.1MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=7f7a0b35da2bad834fd79dd3671061eb04c4503613d5e88b8a4613686403ae4e
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=756948c7b0ef08d3cdca7581250596cb437a3dd862452c05eb715b13663d90ca
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df

### Import Libs

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import gc
import json
import re

import fasttext
import gensim
from gensim.models import FastText as fasttext_gensim
from gensim.test.utils import get_tmpfile

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# from sciosci.assets import keyword_assets as kw
# from sciosci.assets import generic_assets as sci
# from sciosci.assets import advanced_assets as aa

from science_science.sciosci.assets import keyword_assets as kw
from science_science.sciosci.assets import generic_assets as sci
from science_science.sciosci.assets import advanced_assets as aa

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
stops = ['a','an','we','result','however','yet','since','previously','although','propose','proposed','e_g','method',
         'published_elsevier','b','v','problem','paper','approach','within','with','by','via','way','t','case','issue','level','area','system',
         'work','discussed','seen','put','usually','take','make','author','versus','enables','result','research','design','based']
punkts = [' ','','(',')','[',']','{','}','.',',','!','?','<','>','-','_',':',';','\\','/','|','&','%',"'s","`s",'#','$','@']

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = list(set(stopwords.words("english")))+stops+punkts
np.random.seed(50)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Get embeddings from a pre-trained model


### Load Corpus

In [ ]:
period = '2017-2018'
percentile = 97

#### Option A - Load corpus

In [ ]:
directory = datapath+'Corpus/AI 37k/copyr_deflem_stopword_removed_thesaurus May 16/by period/n-gram by 2 repetition keywords/'
file_name = period+' abstract_title'#corpus abstract-title - with n-grams'
corpus = pd.read_csv(directory+file_name,names=['abstracts'])
corpus_tokens = [item.lower() for sublist in corpus['abstracts'].values.tolist() for item in sublist.split()]
gc.collect()

0

In [ ]:
corpus

,abstracts
0,computing SEMANTIC_SIMILARITY based novel mode...
1,ARTIFICIAL_INTELLIGENCE paradigm HEURISTIC_SEA...
2,recognition model driving risk based BELIEF_RU...
3,weighting based SEMANTIC_SIMILARITY_MEASURE ba...
4,insight prediction heterogeneous CARBONATE_RES...
...,...
7800,groundwater VULNERABILITY_INDEX conditioned su...
7801,industrial INFORMATION_INTEGRATION approach or...
7802,ethical concern surviving via robot cyborg exp...
7803,hybrid REGRESSION_MODEL NEAR_REAL_TIME urban w...


#### Option B - Load keywords

In [ ]:
directory = datapath+'LDA/'
file_name = period+' top_90-percentile_keywords_terms.csv'
corpus = pd.read_csv(directory+file_name)
corpus = corpus.fillna('this_is_null')
corpus_tokens = []
for idx,row in tqdm(corpus.iterrows(),total=corpus.shape[0]):
    for token in row.values.tolist():
        if token != 'this_is_null': 
            corpus_tokens.append(token) 
del corpus
print("\nNumber of unique tokens:",len(corpus_tokens))

100%|██████████| 6546/6546 [00:00<00:00, 8418.10it/s]


Number of unique tokens: 52365


#### Option C - Load author keywords

In [ ]:
directory = datapath+'Author keywords - 29 Oct 2019/'
file_name = period+' keyword frequency'
corpus = pd.read_csv(directory+file_name,names=['keyword','frequency'])
corpus = corpus.fillna('this_is_null')
threshold = np.percentile(corpus['frequency'].values,percentile)
corpus = corpus[corpus['frequency']>threshold]

corpus_tokens = []
for idx,row in tqdm(corpus.iterrows(),total=corpus.shape[0]):
    if row['keyword'] != 'this_is_null': 
        corpus_tokens.append(row['keyword']) 
print("\nNumber of unique tokens:",len(corpus_tokens))

100%|██████████| 1376/1376 [00:00<00:00, 9769.37it/s]


Number of unique tokens: 1376


#### Option D - Load benchmark data (labeled)

In [ ]:
directory = datapath+'embedding_benchmark/'
file_name = 'kpris_data.csv'#corpus abstract-title - with n-grams'
corpus = pd.read_csv(directory+file_name)
corpus = corpus.dropna()
corpus_tokens = [item.lower() for sublist in corpus['abstract'].values.tolist() for item in sublist.split()]
gc.collect()

357

## Facebook Model

#### Load model

In [ ]:
fb_model_address = datapath+'/FastText-crawl-300d-2M-subword/crawl-300d-2M-subword.bin'

In [ ]:
model = fasttext.load_model(fb_model_address)

#### Get embeddings

*   Save to dictionary and json
*   This takes much less space on disk


##### No n-gram handle

In [ ]:
# Save in a dict
output_dict = {}
comment_embedding = ''
for token in tqdm(corpus_tokens[:],total=len(corpus_tokens[:])):
    output_dict[token] = str(model.get_word_vector(token))

100%|██████████| 280/280 [00:00<00:00, 337.81it/s]


##### Manual n-gram handle

In [ ]:
# Save in a dict
comment_embedding = 'average_manual '
output_dict = {}
for token in tqdm(corpus_tokens[:],total=len(corpus_tokens[:])):
    token_split = token.split(' ')
    if len(token_split) > 0:
        tmp_vector_grams = []
        for item in token_split:
            tmp_vector_grams.append(model.get_word_vector(item))
        output_dict[token] = str(list(np.array(tmp_vector_grams).mean(axis=0)))
    else:
        output_dict[token] = str(model.get_word_vector(item))

100%|██████████| 280/280 [00:00<00:00, 3792.22it/s]


##### Save to disk

In [ ]:
# Save embeddings to disk
with open(directory+'vectors/100D/FastText vector '+comment_embedding+period+'.json', 'w') as json_file:
    json.dump(output_dict, json_file)

#### Get embeddings (alternative) : 

*   save to a list instead of a dicktionary and csv
*   Will have many redundant words in it and will take lots of disk space




In [ ]:
# Save in a list
batches = 1000
batch_size = len(corpus_tokens)/batches

for step in tqdm(range(batches),total=batches):
    batch_tokens = corpus_tokens[int(step*batch_size):int((step+1)*batch_size)]
    corpus_vectors = [model.get_word_vector(x) for x in batch_tokens]
    corpus_vectors = pd.DataFrame(corpus_vectors)
    corpus_vectors['tokens'] = batch_tokens

    # Save embeddings to disk
    with open(directory+'vector '+period,'a') as f:
        corpus_vectors.to_csv(f,index=False,header=False)

## Gensim Model

#### Load model

Load gensim model

In [ ]:
gensim_model_address = datapath+'embedding_benchmark/clean/fasttext/finetuned_model.model'
model = fasttext_gensim.load(gensim_model_address)

In [ ]:
gensim_model_address = datapath+'FastText Models/50D May 16/fasttext-scopus-300k_docs-gensim 50D.model'
model = fasttext_gensim.load(gensim_model_address)

In [ ]:
gensim_model_address = 'drive/My Drive/Data-Permenant/FastText-crawl-300d-2M-subword/crawl-300d-2M-subword.bin'
model = gensim.models.fasttext.load_facebook_model(gensim_model_address)

Simple Tests

In [ ]:
print('intelligence' in model.wv.vocab)
print(model.similarity("anns", "ann"))
print(model.most_similar(['eye','vision','processing'], ['computer']))
print(model.wmdistance(['stop', 'word', 'removed', 'tokens', 'of', 'sentence 1'], ['stop word removed tokens of sentence 2']))

True
0.87301654
[('viscera', 0.7777436375617981), ('skinvision', 0.7746936678886414), ('visionscan', 0.7712220549583435), ('fingertapping', 0.7613106966018677), ('kinectfusion', 0.7577728033065796), ('visiography', 0.7574525475502014), ('skinnydip', 0.7567591667175293), ('visiondetection', 0.7542468905448914), ('seye', 0.7529548406600952), ('eyetm', 0.7522386312484741)]
21.742491218831038


In [ ]:
# get distance of two words
from scipy import spatial,sparse,sign
vec_a = model.wv['']
vec_b = model.wv['fpga']
distance_tmp = spatial.distance.cosine(vec_a, vec_b)
similarity_tmp = 1 - distance_tmp
similarity_tmp

0.33358505368232727

In [ ]:
model.wv['artificial intelligence']

array([-4.25525   ,  0.03749213,  1.8276842 , -3.1528432 , -3.3440664 ,
        0.66207427,  1.0964861 , -2.038055  ,  3.0331683 , -2.1755521 ,
        2.1063838 ,  1.6578307 ,  1.3311137 , -2.030598  , -0.69794494,
        2.6208954 ,  1.9154872 ,  1.6715113 ,  0.23561044, -0.50721526,
        3.1775064 , -2.069317  , -2.4310536 , -1.8514946 ,  1.3029549 ,
        3.482592  , -2.1535952 ,  1.078043  , -3.8000522 ,  0.08382007,
       -0.6016187 ,  3.3550935 ,  2.5037699 , -2.8812122 , -0.11693893,
       -0.51311666,  3.1224    ,  0.46978405, -0.4427654 , -2.5400903 ,
        2.0880878 ,  3.123557  ,  0.8703581 , -1.0431769 , -2.8512125 ,
        2.2627175 ,  1.0080537 ,  0.1098367 ,  1.5881126 , -1.870272  ],
      dtype=float32)

In [ ]:
(model.wv['artificial']+model.wv['intelligence'])/2

In [ ]:
model.wv['artificial_intelligence']

#### Compare vectors to ACM categories

In [ ]:
AI_categories = [
              'Artificial Intelligence Applications - Expert Systems',
              'Automatic Programming',
              'Deduction - Theorem Proving',
              'Knowledge Representation Formalisms - Knowledge Representation Methods',
              'Programming Languages - Software',
              'Learning',
              'Natural Language Processing',
              'Problem Solving - Control Methods - Search',
              'Robotics',
              'Vision - Scene Understanding',
              'Distributed Artificial Intelligence',
              'ARTIFICIAL INTELLIGENCE'
]

In [ ]:
categories = [
              'Natural language processing - Information extraction - Machine translation - Discourse, dialogue  pragmatics - Natural language generation - Speech recognition - Lexical semantics - Phonology / morphology',
              'Knowledge representation reasoning - Description logics - Semantic networks Nonmonotonic default reasoning  belief revision - Probabilistic reasoning - Vagueness fuzzy logic - Causal reasoning  diagnostics - Temporal reasoning - Cognitive robotics - Ontology engineering - Logic programming answer set programming - Spatial  physical reasoning - Reasoning about belief  knowledge',
              'Planning  scheduling - Planning for deterministic actions - Planning under uncertainty - Multi-agent planning - Planning  abstraction  generalization - Robotic planning - Evolutionary robotics',
              'Search methodologies - Heuristic function construction - Discrete space search - Continuous space search - Randomized search - Game tree search - Abstraction  micro-operators - Search with partial observations - ',
              'Control methods - Robotic planning - Evolutionary robotics - Computational control theory - Motion path planning',
              'Philosophical theoretical foundations artificial intelligence - Cognitive science - Theory mind',
              'Distributed artificial intelligence - Multi agent systems - Intelligent agents - Mobile agents - Cooperation  coordination',
              'Computer vision - Biometrics - Scene understanding - Activity recognition  understanding - Video summarization - Visual content based indexing  retrieval - Visual inspection - Vision for robotics - Scene anomaly detection - Image  video acquisition - Camera calibration - Epipolar geometry - Computational photography - Hyperspectral imaging - Motion capture - 3D imaging - Active vision - Image representations - Shape representations - Appearance  texture - Hierarchical representations - Computer vision problems - Interest point  salient region detections - Image segmentation  - Video segmentation - Shape inference - Object detection - Object recognition - Object identification - Tracking - Reconstruction - Matching',
              
              'Learning paradigms - Supervised learning - Ranking - Learning to rank - Supervised learning  classification - Supervised learning  regression - Structured outputs - Cost sensitive learning - Unsupervised learning - Cluster analysis - Anomaly detection - Mixture modeling - Topic modeling - Source separation - Motif discovery - Dimensionality reduction  manifold learning - Reinforcement learning - Sequential decision making - Inverse reinforcement learning - Apprenticeship learning - Multi-agent reinforcement learning - Adversarial learning - Multi-task learning - Transfer learning - Lifelong machine learning - Learning under covariate shift',
              'Learning settings - Batch learning - Online learning settings - Learning from demonstrations - Learning from critiques - Learning from implicit feedback - Active learning settings - Semi supervised learning settings',
              'Machine learning approaches - Classification  regression trees - Kernel methods - Support vector machines - Gaussian processes - Neural networks - Logical  relational learning - Inductive logic learning - Statistical relational learning - Learning in probabilistic graphical models - Maximum likelihood modeling - Maximum entropy modeling - Maximum a posteriori modeling - Mixture models - Latent variable models - Bayesian network models - Learning linear models - Perceptron algorithm - Factorization methods - Non-negative matrix factorization - Factor analysis - Principal component analysis - Canonical correlation analysis - Latent Dirichlet allocation - Rule learning - Instance-based learning - Markov decision processes -  Markov decision processes - Stochastic games - Learning latent representations - Deep belief networks - Bio inspired approaches - Artificial life - Evolvable hardware - Genetic algorithms - Genetic programming - Evolutionary robotics - Generative  developmental approaches',
              'Machine learning algorithms - Dynamic programming Markov decision processes - Value iteration - Q learning - Policy iteration - Temporal difference learning - Approximate dynamic programming methods - Ensemble methods - Boosting - Bagging - Spectral methods - Feature selection - Regularization',
]

In [ ]:
AI_vectors = []
labels = []
for item in categories:
    vector_tmp = []
    label = item.split('-')[0]
    for phrase in item.split('-'):
        phrase = phrase.lower().strip()
        # print(phrase)
        vector_tmp.append(model.wv[phrase])
    # print('---')
    AI_vectors.append(list(np.array(vector_tmp).mean(axis=0)))
    labels.append(label)
print(AI_vectors)

[[-4.2585297, -1.09474, -0.51740396, -1.5583799, -1.277385, 1.2230147, 3.6439853, -1.6600533, 1.4912666, -2.3865879, 1.2189151, 0.5719612, 2.7518144, -2.781408, -2.061385, 2.1494553, 1.0412952, 2.237492, 0.14161092, -1.4961629, 1.8666816, -1.0945007, -0.40818986, -1.4722152, -0.20459145, 2.328488, -0.7185293, 0.63060343, -2.3967595, -1.0597452, 1.1229446, 1.805113, 1.015998, -2.1873384, -0.12749422, -0.016316757, 1.9769442, 0.9840427, -1.7754203, -3.5659814, 3.1010442, 2.0076227, 0.61076725, -0.32515174, -3.6688373, -1.054969, 2.3143127, 0.7147262, 1.1658139, 0.8725452], [-4.7960773, -1.7704331, -0.17159314, -1.5654887, -1.696463, 1.3633636, 2.3642745, -0.60040635, 1.4829278, -2.4751508, 1.0201765, 0.6441909, 2.688061, -2.4529297, 0.47901928, 2.3566878, 1.1981105, 0.46998572, 0.58817506, -0.64123666, 2.3364296, -2.3828585, 0.27646962, -0.96989816, 0.2530441, 1.0085167, -0.6223591, 0.35445678, -2.2382672, -0.57872456, 1.659824, 2.0099118, 2.595362, -0.9997726, -0.09018091, 0.8816102, 2.

In [ ]:
pd.DataFrame(AI_vectors).to_csv(datapath+'FastText doc clusters - SIP/50D/classification/ACM_classifications_vectors')
pd.DataFrame(labels,columns=['label']).to_csv(datapath+'FastText doc clusters - SIP/50D/classification/ACM_classifications_labels')

### Get Word Embeddings

##### No n-gram handle

In [ ]:
# Save in a dict
comment_embedding = ''
output_dict = {}
for token in tqdm(corpus_tokens[:],total=len(corpus_tokens[:])):
    output_dict[token] = str(model.wv[token])

##### Manual n-gram handle

In [ ]:
# Save in a dict
comment_embedding = 'average_manual '
output_dict = {}
for token in tqdm(corpus_tokens[:],total=len(corpus_tokens[:])):
    token_split = token.split(' ')
    if len(token_split) > 0:
        tmp_vector_grams = []
        for item in token_split:
            tmp_vector_grams.append(model.wv[item])
        output_dict[token] = str(list(np.array(tmp_vector_grams).mean(axis=0)))
    else:
        output_dict[token] = str(model.wv[token])

100%|██████████| 1590/1590 [00:00<00:00, 10574.09it/s]


##### Save to disk

In [ ]:
# Save embeddings to disk
with open(directory+'/FastText vector with n-grams '+comment_embedding+period+'.json', 'w') as json_file:
    json.dump(output_dict, json_file)

In [ ]:
print(directory)

drive/My Drive/Data/corpus/improved_copyr_lemmatized_stopwords_removed_thesaurus_n-grams/


### Get Doc Embeddings (SIF)

It is not recommended to perform this on cloud, as it is not process intesive, yet takes too long depending on the doc-count. It might take over 30 hours.

Make a probability dictionary

In [ ]:
corpus_tokens_s = pd.Series(corpus_tokens)
corpus_tokens_probabilities = (corpus_tokens_s.groupby(corpus_tokens_s).transform('count') / len(corpus_tokens_s)).values
corpus_tokens_probabilities = pd.DataFrame(corpus_tokens_probabilities)
corpus_tokens_probabilities['tokens'] = corpus_tokens_s
corpus_tokens_probabilities.columns = ['probability','token']
corpus_tokens_probabilities = corpus_tokens_probabilities.groupby('token').mean()
corpus_tokens_probabilities = corpus_tokens_probabilities.reset_index()
corpus_tokens_probabilities.columns = ['token','probability']

Get vectors

In [ ]:
vectors = []
for token in tqdm(corpus_tokens_probabilities['token'],total=corpus_tokens_probabilities.shape[0]):
    vectors.append(model.wv[token])
corpus_tokens_probabilities['vector'] = vectors


100%|██████████| 41902/41902 [00:01<00:00, 23568.01it/s]


Calculate weighted average vectors

In [ ]:
a = 1e-3
embedding_size = 50

doc_set = []
for doc in tqdm(corpus['abstract'].values.tolist(),total=len(corpus['abstract'].values.tolist())):
    vs = np.zeros(embedding_size)  # add all word2vec values into one vector for the sentence
    doc_length = len(doc.split())
#     print(doc.split())
    for word in doc.lower().split():
        a_value = a / (a + corpus_tokens_probabilities[corpus_tokens_probabilities['token']==word]['probability'].values.tolist()[0])  # smooth inverse frequency, SIF
        vs = np.add(vs, np.multiply(a_value, corpus_tokens_probabilities[corpus_tokens_probabilities['token']==word]['vector'].values.tolist()[0]))  # vs += sif * word_vector

    vs = np.divide(vs, doc_length)  # weighted average
    doc_set.append(vs)  # add to our existing re-calculated set of sentences



 16%|█▌        | 3077/19710 [1:36:36<5:27:17,  1.18s/it]

In [ ]:
pd.DataFrame(doc_set).to_csv(datapath+'embedding_benchmark/Document Embedding/finetuned_embeddings.csv',index=False)

##### Wikipedia embedding

In [ ]:
import os
import time
from stat import S_ISREG, ST_CTIME, ST_MODE

print("\nSearching for Wiki texts...\n")
dir_path = datapath+'Corpus/AI Wiki Classifications/applications/clean/'
data = (os.path.join(dir_path, fn) for fn in os.listdir(dir_path))
data = ((os.stat(path), path) for path in data)
data = ((stat[ST_CTIME], path) for stat, path in data if S_ISREG(stat[ST_MODE]))

names = []
files = []
for cdate, path in sorted(data):
    print('   - ', time.ctime(cdate), os.path.basename(path),int(os.path.getsize(path)/1000000),'MB')
    files.append(path)
    names.append(os.path.basename(path))


Searching for Wiki texts...

   -  Tue May 26 08:30:59 2020 Artificial life 0 MB
   -  Tue May 26 08:30:59 2020 Automated planning and scheduling 0 MB
   -  Tue May 26 08:30:59 2020 Computational creativity 0 MB
   -  Tue May 26 08:31:00 2020 Automated reasoning 0 MB
   -  Tue May 26 08:31:03 2020 Automation 0 MB
   -  Tue May 26 08:31:04 2020 Automatic target recognition 0 MB
   -  Tue May 26 08:31:05 2020 Bio-inspired computing 0 MB
   -  Tue May 26 08:31:05 2020 Computer audition 0 MB
   -  Tue May 26 08:31:06 2020 Diagnosis (artificial intelligence) 0 MB
   -  Tue May 26 08:31:07 2020 Expert system 0 MB
   -  Tue May 26 08:31:09 2020 Decision support system 0 MB
   -  Tue May 26 08:31:15 2020 Computer vision and subfields 0 MB
   -  Tue May 26 08:31:17 2020 Computer vision 0 MB
   -  Tue May 26 08:31:19 2020 Artificial intelligence in video games 0 MB
   -  Tue May 26 08:31:20 2020 Hybrid intelligent system 0 MB
   -  Tue May 26 08:31:21 2020 Intelligent agent 0 MB
   -  Tue May 2

In [ ]:
embedding_size = 50
all = []
for file_index,file in enumerate(files):
    print('\n',file,'\n')
    corpus = pd.read_csv(file)
    corpus_tokens = [item.lower() for sublist in corpus['sentence'].values.tolist() if pd.notnull(sublist) for item in sublist.split()]
    gc.collect()
    
    corpus_tokens_s = pd.Series(corpus_tokens)
    corpus_tokens_probabilities = (corpus_tokens_s.groupby(corpus_tokens_s).transform('count') / len(corpus_tokens_s)).values
    corpus_tokens_probabilities = pd.DataFrame(corpus_tokens_probabilities)
    corpus_tokens_probabilities['tokens'] = corpus_tokens_s
    corpus_tokens_probabilities.columns = ['probability','token']
    corpus_tokens_probabilities = corpus_tokens_probabilities.groupby('token').mean()
    corpus_tokens_probabilities = corpus_tokens_probabilities.reset_index()
    corpus_tokens_probabilities.columns = ['token','probability']

    vectors = []
    for token in tqdm(corpus_tokens_probabilities['token'],total=corpus_tokens_probabilities.shape[0]):
        vectors.append(model.wv[token])
    corpus_tokens_probabilities['vector'] = vectors

    a = 1e-3

    doc_set = []
    for doc in tqdm(corpus['sentence'].values.tolist(),total=len(corpus['sentence'].values.tolist())):
        vs = np.zeros(embedding_size)  # add all word2vec values into one vector for the sentence
        if pd.notnull(doc):
            doc_length = len(doc.split())
        #     print(doc.split())
            for word in doc.lower().split():
                a_value = a / (a + corpus_tokens_probabilities[corpus_tokens_probabilities['token']==word]['probability'].values.tolist()[0])  # smooth inverse frequency, SIF
                vs = np.add(vs, np.multiply(a_value, corpus_tokens_probabilities[corpus_tokens_probabilities['token']==word]['vector'].values.tolist()[0]))  # vs += sif * word_vector

            vs = np.divide(vs, doc_length)  # weighted average
            doc_set.append(vs)  # add to our existing re-calculated set of sentences

    pd.DataFrame(doc_set).to_csv(datapath+'Corpus/AI Wiki Classifications/applications/clean/vectors/'+names[file_index],index=False)

    all.append(pd.DataFrame(doc_set).mean(axis=0))

all_df = pd.DataFrame(all)
all_df['clusters'] = names
all_df.to_csv(datapath+'Corpus/AI Wiki Classifications/applications/clean/vectors/all',index=False)


 drive/My Drive/Data/Corpus/AI Wiki Classifications/applications/clean/Artificial life 



100%|██████████| 91/91 [00:01<00:00, 80.89it/s]



 drive/My Drive/Data/Corpus/AI Wiki Classifications/applications/clean/Automated planning and scheduling 



100%|██████████| 107/107 [00:01<00:00, 93.22it/s]



 drive/My Drive/Data/Corpus/AI Wiki Classifications/applications/clean/Computational creativity 



...

100%|██████████| 714/714 [00:12<00:00, 58.94it/s]



 drive/My Drive/Data/Corpus/AI Wiki Classifications/applications/clean/Computer vision 



100%|██████████| 281/281 [00:04<00:00, 66.51it/s]



 drive/My Drive/Data/Corpus/AI Wiki Classifications/applications/clean/Artificial intelligence in video games 



  0%|          | 0/869 [00:00<?, ?it/s]

# Train on a large Scopus corpus


#### Load Corpus Sentences

In [ ]:
sentence_corpus = pd.read_csv(datapath+'embedding_benchmark/clean/corpus sentences abstract-title')

In [ ]:
sentence_corpus['sentence'][812]

'in still other exa'

#### Preprocess and prepare corpus for FastText training

In [ ]:
sentences = []
lemmatizer=WordNetLemmatizer()
with open(datapath+'embedding_benchmark/clean/corpus sentences abstract-title', 'w') as f:
    for index,row in tqdm(sentence_corpus.iterrows(),total=sentence_corpus.shape[0]):
        sentence = row['sentence']
        sentence = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",sentence)
        sentence = word_tokenize(sentence)
        sentence = [word for word in sentence if not word in punkts] 
        sentence = [lemmatizer.lemmatize(word) for word in sentence]
        # sentences.append(sentence)
        f.write("%s\n" % ' '.join(sentence))

#### Save sentences to disk for future use -- Not needed anymore

In [ ]:
pd.DataFrame([' '.join(words) for words in sentences],columns=['sentences']).to_csv(
    datapath+'corpus/AI ALL/1900-2019 corpus sentences abstract-title further processed.csv',
    header=True,index=False)

#### Load pre-processed sentences

In [ ]:
sentence_corpus = pd.read_csv(datapath+'corpus/AI ALL/1900-2019 corpus sentences abstract-title further processed.csv',delimiter=";;;")
sentence_corpus.columns = ["sentences"]
sentences = []
sentence_corpus = sentence_corpus.fillna('')
for index,row in tqdm(sentence_corpus.iterrows(),total=sentence_corpus.shape[0]):
    sentences.append(row['sentences'].split(' '))

100%|██████████| 13230009/13230009 [28:30<00:00, 7736.00it/s]


In [ ]:
sentence_corpus.head(10)

,article_index,sentence
0,0,a lighting system is disclosed comprising a pl...
1,0,the first image projection lighting device is ...
2,0,the camera of the first image projection light...
3,1,a photographic device eliminates photometric e...
4,1,the device includes an orientation position de...
5,1,a photometre divides the subject field into mu...
6,1,an exposure calculator calculates the exposure...
7,1,a memory stores weighing coefficients correspo...
8,1,a tilt switch posi
9,2,a print on demand camera system in which the c...


### Train Fasttext - Gensim

#### Load a model to continue training

* If want to continue training, run this section

In [ ]:
gensim_model_address = datapath+'FastText Models/50D May 16/fasttext-scopus-300k_docs-gensim 50D.model'

In [ ]:
model = fasttext_gensim.load(gensim_model_address)

In [ ]:
sentence_corpus = sentence_corpus.dropna()
sentences = sentence_corpus.sentence.values.tolist()
# sentences

In [ ]:
model.build_vocab(sentences, update=True)
model.train(sentences, total_examples=len(sentences), epochs=model.epochs)

* Otherwise run this section

#### Train

In [ ]:
model = fasttext_gensim(min_n=3, max_n=6, size=15, window=5, min_count=1, seed = 50)
model.build_vocab(sentences=sentences)
model.train(sentences=sentences, total_examples=len(sentences), epochs=10)

In [ ]:
fname = datapath+'embedding_benchmark/clean/fasttext/finetuned_model.model'
model.save(fname)

In [ ]:
model_new = fasttext_gensim(min_n=3, max_n=6, size=50, window=5, min_count=1, seed = 50)
model_new.build_vocab(sentences=sentences)
model_new.train(sentences=sentences, total_examples=len(sentences), epochs=20)
fname = datapath+'embedding_benchmark/clean/fasttext/trained_model.model'
model_new.save(fname)

In [ ]:
model = fasttext_gensim(min_n=3, max_n=6, size=100, window=5, min_count=1, seed = 50)
model.build_vocab(sentences=sentences)
model.train(sentences=sentences, total_examples=len(sentences), epochs=10)
fname = datapath+"Models/fasttext-scopus-2.2-million_docs-gensim 50D.model"
model.save(fname)

#### Test model

In [ ]:
similarities = model.wv.most_similar(positive=['logic','fuzzy','expert'],negative=['deep','neural','network','cnn','ann'])
most_similar = similarities[0]

In [ ]:
most_similar

('mam-rnn', 0.9763791561126709)

In [ ]:
not_matching = model.wv.doesnt_match("human computer interface tree".split())

In [ ]:
not_matching

'tree'

In [ ]:
sim_score = model.wv.similarity('computer', 'human')

In [ ]:
sim_score

0.7571839

In [ ]:
print(model.wv['artificial intelligence'])
print(model.wv['artificial'])
print(model.wv['intelligence'])

[ 1.2783480e+00 -4.2018552e+00  7.1276689e-01  4.2023015e+00
 -5.0359420e-03  4.4385982e+00  6.2421050e+00 -8.9032326e+00
  1.7556003e+00  1.3425230e+00  9.4295764e-01 -4.4485557e-01
 -5.8648558e+00  2.6428668e+00 -1.2076639e+00]
[  3.7072854   -3.616749     1.3040072    0.234361    -2.753659
   7.528801    14.293305   -14.688236     5.3885765    6.496681
   1.9917868    2.855616    -0.05153261   7.8660994   -2.22459   ]
[  0.28606984  -6.971052    -0.9232919   11.48035      0.2561571
   4.084776     2.4220266   -8.616226     0.94255084  -2.2498865
   1.7112938   -3.370861   -12.577294    -1.1608386   -0.04991044]


### Train Fasttext - Facebook

In [ ]:
sentences_joined = ' '.join(sentences)
model = fasttext.train_unsupervised(sentences_joined, "cbow", minn=2, maxn=5, dim=50, epoch=10,lr=0.05)

#### Test model

In [ ]:
model.words

In [ ]:
model.get_word_vector("the")

In [ ]:
model.get_nearest_neighbors('asparagus')

In [ ]:
model.get_analogies("intelligence", "math", "fuzzy")

#### Save model

In [ ]:
model.save_model(datapath+"fasttext-scopus_wos-merged-310k_docs-facebook.ftz")